In [1]:
!pip install -U keras-nlp
!pip install -U keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: keras-nlp
    Found existing installation: keras-nlp 0.7.0
    Uninstalling keras-nlp-0.7.0:
      Successfully uninstalled keras-nlp-0.7.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.3/381.3 kB 23.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.6.0 which is incompatible.
tensorflowjs 4.16.0 requires packaging~=23.1, but you have packaging 21.3 which is incompatible

In [2]:
import os
import pandas as pd
import numpy as np

import keras
import keras_nlp

2024-11-22 16:12:07.877956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-22 16:12:07.878101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-22 16:12:08.064918: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import random
random.seed(123)

In [4]:
# adjust display options
pd.set_option('display.max_colwidth', None)

In [5]:
# load submission file = input data
df = pd.read_csv('../input/santa-2024/sample_submission.csv')
n_rows = df.shape[0]
df

,id,text
0,0,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge
1,1,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and
2,2,yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice
3,3,yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap
4,4,hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle
5,5,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle


In [6]:
# setting up model
os.environ['KERAS_BACKEND'] = 'jax'
keras.config.set_floatx('bfloat16')

# load model
LLM = keras_nlp.models.CausalLM.from_preset('gemma_instruct_2b_en')

# show model details
LLM.summary()

Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'preprocessor.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (4.67 GB)

 Trainable params: 2,506,172,416 (4.67 GB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# define prompt
prompt_base = 'Please create a permutation of the words in the following input text in order to get a lower perplexity. Please make sure to have exactly the same words in the output as in the input. Please calculate the perplexity for the input and the output, explain how the calculation was done, and compare them. Here is the input text:\n'
print(prompt_base)

Please create a permutation of the words in the following input text in order to get a lower perplexity. Please make sure to have exactly the same words in the output as in the input. Please calculate the perplexity for the input and the output, explain how the calculation was done, and compare them. Here is the input text:



In [8]:
# evaluate for each text
for i in range(0, n_rows):
    print('> Text #', i+1)
    text = df.text[i]
    my_prompt = prompt_base + text
    response = LLM.generate(my_prompt)
    print('Response:', response)
    print('\n\n\n')

> Text # 1


I0000 00:00:1732292011.465706      26 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Response: Please create a permutation of the words in the following input text in order to get a lower perplexity. Please make sure to have exactly the same words in the output as in the input. Please calculate the perplexity for the input and the output, explain how the calculation was done, and compare them. Here is the input text:
advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge

The output permutation is:

- chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge

The perplexity for the input is 12.2. The perplexity for the output is 10.5.

**Explanation:**

The difference in perplexity between the input and the output is due to the order of the words in the input. The words in the output are arranged in a different order than the words in the input. This changes the way that the words are interpreted by the model, which can lead to a lower perplexity.

**Comparison:**

The perplexity of the input is 12.2, while the perplexity

#### Very entertaining but full of mistakes...

In [9]:
# ask for Python code snippet
response = LLM.generate('Please generate a python code snippet which can be used to change the order of words randomly in an input text.')
print(response)

Please generate a python code snippet which can be used to change the order of words randomly in an input text.

```python
# This Python code snippet can be used to change the order of words randomly in an input text.

import random

def random_word_order(text):
  """
  Randomly orders the words in a text.

  Args:
    text: The text to randomize.

  Returns:
    The randomized text.
  """

  # Split the text into a list of words.
  words = text.split()

  # Shuffle the words in the list.
  random.shuffle(words)

  # Join the words back together with a space.
  random_text = ' '.join(words)

  return random_text


# Get the input text from the user.
input_text = input("Enter a text: ")

# Randomly order the words in the input text.
random_text = random_word_order(input_text)

# Print the randomized text.
print("Randomized text:", random_text)
```

**Example Usage:**

```
Enter a text: This is a sample text with some words in a random order.
```

**Output:**

```
Randomized text: Some w

In [10]:
import random

def random_word_order(text):

  # Split the text into a list of words.
  words = text.split()

  # Shuffle the words in the list.
  random.shuffle(words)

  # Join the words back together with a random separator.
  random_text = ' '.join(words)

  return random_text

In [11]:
# apply function
df['text_shuffled'] = df.text.map(random_word_order)

In [12]:
# show result
df

,id,text,text_shuffled
0,0,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge,advent chimney elf family ornament fireplace gingerbread scrooge reindeer mistletoe
1,1,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and,drive night jump and family reindeer advent mistletoe scrooge laugh the fireplace walk ornament bake elf gingerbread chimney sleep give
2,2,yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice,nutcracker beard grinch jingle decorations stocking workshop holiday holly sleigh polar naughty cheer carol ornament nice chimney gifts magi yuletide
3,3,yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap,gifts cheer and cheer holly visit nutcracker carol jingle unwrap grinch eat nice sing sleigh stocking naughty chimney polar of magi is holiday yuletide decorations relax the workshop ornament beard
4,4,hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle,believe night peace snowglobe fireplace workshop hohoho cookie fruitcake wreath we wrapping wonder eggnog candy joy have to as wish star hope toy it in not you card chocolate and doll puzzle kaggle of game from greeting bow season the poinsettia milk dream merry candle paper that with angel peppermint
5,5,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle,gifts holiday eat fireplace scrooge bow beard mistletoe elf eggnog cheer ornament grinch that candy cookie wrapping game card not in laugh workshop the night peace nutcracker hope advent unwrap and and poinsettia nice fireplace sing snowglobe sleep fruitcake carol merry sleigh kaggle is reindeer paper holly decorations candle and you angel bake jingle wish puzzle chimney as relax toy workshop jump doll walk wreath gingerbread hohoho stocking to magi chimney milk ornament give star greeting polar visit joy drive wonder of night season believe of cheer dream yuletide family peppermint naughty we the have it from with chocolate the


In [13]:
# ask for another Python code snippet
response = LLM.generate('Please generate a python code snippet that just transposes two words randomly in an input text.')
print(response)

Please generate a python code snippet that just transposes two words randomly in an input text.

```python
import random

text = input("Enter a text: ")

# Split the text into a list of words
words = text.split()

# Randomly select two words from the list
word1 = random.choice(words)
word2 = random.choice(words)

# Swap the positions of the two words in the list
words[words.index(word1)] = word2
words[words.index(word2)] = word1

# Join the words back into a string
transposed_text = " ".join(words)

# Print the transposed text
print("Transposed text:", transposed_text)
```

**Example Usage:**

```
Enter a text: This is a sample text. It has multiple words.
```

**Output:**

```
Transposed text: It is a sample text. It has multiple words.
```

**Explanation:**

1. The code first imports the `random` module for random word selection.
2. It then takes the input text and splits it into a list of words using the `split()` method.
3. The code then randomly selects two words from the list usi

In [14]:
def transpose_words(text):
    
    # Split the text into a list of words
    words = text.split()

    # Randomly select two words from the list
    word1 = random.choice(words)
    # just a small change to avoid trivial transpositions:
    # word2 = random.choice(words)
    words_left = words.copy()
    words_left.remove(word1)
    word2 = random.choice(words_left)
    print(word1, '<->' , word2)
    # Swap the positions of the two words in the list
    words_new = words.copy() # yet another manual fix...
    words_new[words.index(word1)] = word2
    words_new[words.index(word2)] = word1
    
    # Join the words back into a string
    transposed_text = " ".join(words_new)

    return transposed_text

In [15]:
# apply function
df['text_transposition'] = df.text.map(transpose_words)

elf <-> scrooge
jump <-> give
gifts <-> sleigh
cheer <-> grinch
wish <-> in
cheer <-> ornament


In [16]:
# show results
df

,id,text,text_shuffled,text_transposition
0,0,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge,advent chimney elf family ornament fireplace gingerbread scrooge reindeer mistletoe,advent chimney scrooge family fireplace gingerbread mistletoe ornament reindeer elf
1,1,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and,drive night jump and family reindeer advent mistletoe scrooge laugh the fireplace walk ornament bake elf gingerbread chimney sleep give,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk jump give drive bake the sleep night laugh and
2,2,yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice,nutcracker beard grinch jingle decorations stocking workshop holiday holly sleigh polar naughty cheer carol ornament nice chimney gifts magi yuletide,yuletide decorations sleigh cheer holiday carol magi nutcracker polar grinch gifts chimney workshop stocking ornament holly jingle beard naughty nice
3,3,yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap,gifts cheer and cheer holly visit nutcracker carol jingle unwrap grinch eat nice sing sleigh stocking naughty chimney polar of magi is holiday yuletide decorations relax the workshop ornament beard,yuletide decorations gifts grinch holiday carol magi nutcracker polar cheer sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap
4,4,hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle,believe night peace snowglobe fireplace workshop hohoho cookie fruitcake wreath we wrapping wonder eggnog candy joy have to as wish star hope toy it in not you card chocolate and doll puzzle kaggle of game from greeting bow season the poinsettia milk dream merry candle paper that with angel peppermint,hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star in wreath angel the to of and wish that have it not with as you from we kaggle
5,5,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle,gifts holiday eat fireplace scrooge bow beard mistletoe elf eggnog cheer ornament grinch that candy cookie wrapping game card not in laugh workshop the night peace nutcracker hope advent unwrap and and poinsettia nice fireplace sing snowglobe sleep fruitcake carol merry sleigh kaggle is reindeer paper holly decorations candle and you angel bake jingle wish puzzle chimney as relax toy workshop jump doll walk wreath gingerbread hohoho stocking to magi chimney milk ornament give star greeting polar visit joy drive wonder of night season believe of cheer dream yuletide family peppermint naughty we the have it from with chocolate the,advent chimney elf family fireplace gingerbread